# Coursera Capstone project

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

You should consider upgrading via the 'c:\users\marta\anaconda3\python.exe -m pip install --upgrade pip' command.


Libraries imported.


### 1. Introduction/Business Problem

The purpose of this project is to recommend a type of restaurant and area where to open it in the city center of Madrid. 

### 2. Data that will be used

The data I am going to use is a list of zip codes of the city center that i have and then I will add the latitude and longitude. Besides I am going to get the venues data from Foursquare to know the categories of restaurants and cluster the area. Finally I will look for the type missing in each cluster.

Import the data of cps of Madrid

In [2]:
df=pd.read_excel("CPS_MADRID.xlsx")
df.head()

,CP
0,28001
1,28002
2,28003
3,28004
4,28005


Here I get the latitude and longitude

In [3]:
import geocoder

df['Latitude'] = df.apply(lambda row: geocoder.arcgis('{}, Spain'.format(row['CP'])).latlng[0], axis=1)
df['Longitude'] = df.apply(lambda row: geocoder.arcgis('{}, Spain'.format(row['CP'])).latlng[1], axis=1)

In [4]:
df.head()

,CP,Latitude,Longitude
0,28001,40.424870,-3.682793
1,28002,40.445035,-3.673343
2,28003,40.442985,-3.705943
3,28004,40.424445,-3.699007
4,28005,40.405755,-3.712972


Then I get the data from Foursquare

In [5]:
CLIENT_ID = 'RRSN0LZMUFW42FPOHT4UDESOBZE2OEMID2G45EN5YMYIIJ2K' # your Foursquare ID
CLIENT_SECRET = '1OK1F0EQXYJJMLDJG5QMMFVXF4HPLVVCXKHRZLCIK2WCZ4FG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RRSN0LZMUFW42FPOHT4UDESOBZE2OEMID2G45EN5YMYIIJ2K
CLIENT_SECRET:1OK1F0EQXYJJMLDJG5QMMFVXF4HPLVVCXKHRZLCIK2WCZ4FG


In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

madrid_venues = getNearbyVenues(names=df['CP'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])

In [8]:
madrid_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,28001,40.42487,-3.682793,La Cocina De Maria Luisa,40.423513,-3.681738,Spanish Restaurant
1,28001,40.42487,-3.682793,Amazónico,40.423838,-3.685025,Restaurant
2,28001,40.42487,-3.682793,MORE,40.423884,-3.683809,Restaurant
3,28001,40.42487,-3.682793,Federica & Co.,40.426082,-3.684854,Thrift / Vintage Store
4,28001,40.42487,-3.682793,Numa,40.423327,-3.683640,Italian Restaurant


Select only the restaurants

In [9]:
madrid_restaurants=madrid_venues[madrid_venues['Venue Category'].str.contains('Restaurant')]
madrid_restaurants.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,28001,40.42487,-3.682793,La Cocina De Maria Luisa,40.423513,-3.681738,Spanish Restaurant
1,28001,40.42487,-3.682793,Amazónico,40.423838,-3.685025,Restaurant
2,28001,40.42487,-3.682793,MORE,40.423884,-3.683809,Restaurant
4,28001,40.42487,-3.682793,Numa,40.423327,-3.683640,Italian Restaurant
7,28001,40.42487,-3.682793,Swagat Indian Restaurant,40.424513,-3.683114,Indian Restaurant


Let´s how many venues of each type we have in the city

In [10]:
summary_categories=madrid_restaurants[['Venue','Venue Category']]
summary_categories=summary_categories.groupby('Venue Category').count()
summary_categories

,Venue
Venue Category,
American Restaurant,6
Argentinian Restaurant,5
Asian Restaurant,14
Brazilian Restaurant,1
Chinese Restaurant,10
Cuban Restaurant,1
Dumpling Restaurant,2
Falafel Restaurant,1
Fast Food Restaurant,12


Clustering the neighbourhoods

In [11]:
# set number of clusters
kclusters = 6

madrid_CP_clustering = df.drop('CP', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(madrid_CP_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 2, 4, 4, 4, 4, 0, 4, 4, 2, 0, 4, 1,
       2, 2, 2, 4, 4, 4, 0, 1, 1, 1, 5, 0, 0, 0, 1, 4, 0, 2, 4, 5, 5, 2,
       4, 0, 2, 0, 0, 5, 1, 1, 4, 2, 5, 3, 3, 3, 4, 5, 0, 3, 3])

In [13]:
df['Cluster_number']=kmeans.labels_
df.head()


,CP,Latitude,Longitude,Cluster_number
0,28001,40.424870,-3.682793,4
1,28002,40.445035,-3.673343,4
2,28003,40.442985,-3.705943,4
3,28004,40.424445,-3.699007,4
4,28005,40.405755,-3.712972,4


In [14]:
madrid_restaurants.rename(columns={'Neighborhood': 'CP'}, inplace=True)
madrid_restaurants.set_index('CP')


C:\Users\marta\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
CP,,,,,,
28001,40.424870,-3.682793,La Cocina De Maria Luisa,40.423513,-3.681738,Spanish Restaurant
28001,40.424870,-3.682793,Amazónico,40.423838,-3.685025,Restaurant
28001,40.424870,-3.682793,MORE,40.423884,-3.683809,Restaurant
28001,40.424870,-3.682793,Numa,40.423327,-3.683640,Italian Restaurant
28001,40.424870,-3.682793,Swagat Indian Restaurant,40.424513,-3.683114,Indian Restaurant
28001,40.424870,-3.682793,Ultramarinos Quintin,40.423884,-3.685403,Restaurant
28001,40.424870,-3.682793,El Paraguas,40.423865,-3.685814,Spanish Restaurant
28001,40.424870,-3.682793,Joselito´s,40.424874,-3.683982,Restaurant
28001,40.424870,-3.682793,El Pimiento Verde,40.423972,-3.681602,Restaurant


In [15]:
df.set_index('CP')
madrid_restaurants_clustered=pd.merge(df,madrid_restaurants, on='CP')
madrid_restaurants_clustered=madrid_restaurants_clustered.drop(['Neighborhood Latitude','Neighborhood Longitude'],1)

In [16]:
# create map
map_clusters = folium.Map(location=[40.424870, -3.682793], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [57]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(madrid_restaurants_clustered['Latitude'], madrid_restaurants_clustered['Longitude'], madrid_restaurants_clustered['CP'], madrid_restaurants_clustered['Cluster_number']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [17]:
options_restaurant=madrid_restaurants_clustered.groupby(['Cluster_number','Venue Category']).count()
options_restaurant

CP  Latitude  Longitude  Venue  \
Cluster_number Venue Category                                                   
0              American Restaurant              1         1          1      1   
               Asian Restaurant                 2         2          2      2   
               Fast Food Restaurant             4         4          4      4   
               Indian Restaurant                2         2          2      2   
               Italian Restaurant               6         6          6      6   
               Japanese Restaurant              4         4          4      4   
               Kebab Restaurant                 1         1          1      1   
               Mediterranean Restaurant         2         2          2      2   
               Mexican Restaurant               2         2          2      2   
               Peruvian Restaurant              1         1          1      1   
               Restaurant                      17        17         17     17   
               Seafood Restaurant               5         5          5      5   
               Spanish Restaurant              27        27         27     27   
               Sushi Restaurant                 2         2          2      2   
               Tapas Restaurant                 9         9          9      9   
               Thai Restaurant                  1         1          1      1   
               Vegetarian / Vegan Restaurant    1         1          1      1   
1              Asian Restaurant                 1         1          1      1   
               Restaurant                       1         1          1      1   
               Spanish Restaurant               3         3          3      3   
2              Argentinian Restaurant           1         1          1      1   
               Asian Restaurant                 1         1          1      1   
               Chinese Restaurant               1         1          1      1   
               Fast Food Restaurant             2         2          2      2   
               Italian Restaurant               2         2          2      2   
               Paella Restaurant                2         2          2      2   
               Restaurant                      10        10         10     10   
               Seafood Restaurant               1         1          1      1   
               Spanish Restaurant               8         8          8      8   
               Tapas Restaurant                 7         7          7      7   
4              American Restaurant              4         4          4      4   
               Argentinian Restaurant           4         4          4      4   
               Asian Restaurant                 7         7          7      7   
               Brazilian Restaurant             1         1          1      1   
               Chinese Restaurant               9         9          9      9   
               Cuban Restaurant                 1         1          1      1   
               Dumpling Restaurant              2         2          2      2   
               Falafel Restaurant               1         1          1      1   
               Fast Food Restaurant             4         4          4      4   
               French Restaurant                3         3          3      3   
               Greek Restaurant                 3         3          3      3   
               Indian Restaurant                5         5          5      5   
               Italian Restaurant              20        20         20     20   
               Japanese Restaurant             13        13         13     13   
               Korean Restaurant                1         1          1      1   
               Latin American Restaurant        1         1          1      1   
               Mediterranean Restaurant        21        21         21     21   
               Mexican Restaurant              10        10         10     10 

In [18]:
options_restaurant=options_restaurant.drop(['CP','Latitude','Longitude','Venue Latitude','Venue Longitude'],1)
options_restaurant

Venue
Cluster_number Venue Category                      
0              American Restaurant                1
               Asian Restaurant                   2
               Fast Food Restaurant               4
               Indian Restaurant                  2
               Italian Restaurant                 6
               Japanese Restaurant                4
               Kebab Restaurant                   1
               Mediterranean Restaurant           2
               Mexican Restaurant                 2
               Peruvian Restaurant                1
               Restaurant                        17
               Seafood Restaurant                 5
               Spanish Restaurant                27
               Sushi Restaurant                   2
               Tapas Restaurant                   9
               Thai Restaurant                    1
               Vegetarian / Vegan Restaurant      1
1              Asian Restaurant                   1
               Restaurant                         1
               Spanish Restaurant                 3
2              Argentinian Restaurant             1
               Asian Restaurant                   1
               Chinese Restaurant                 1
               Fast Food Restaurant               2
               Italian Restaurant                 2
               Paella Restaurant                  2
               Restaurant                        10
               Seafood Restaurant                 1
               Spanish Restaurant                 8
               Tapas Restaurant                   7
4              American Restaurant                4
               Argentinian Restaurant             4
               Asian Restaurant                   7
               Brazilian Restaurant               1
               Chinese Restaurant                 9
               Cuban Restaurant                   1
               Dumpling Restaurant                2
               Falafel Restaurant                 1
               Fast Food Restaurant               4
               French Restaurant                  3
               Greek Restaurant                   3
               Indian Restaurant                  5
               Italian Restaurant                20
               Japanese Restaurant               13
               Korean Restaurant                  1
               Latin American Restaurant          1
               Mediterranean Restaurant          21
               Mexican Restaurant                10
               Middle Eastern Restaurant          2
               Paella Restaurant                  2
               Persian Restaurant                 1
               Peruvian Restaurant                1
               Portuguese Restaurant              1
               Ramen Restaurant                   1
               Restaurant                        73
               Seafood Restaurant                13
               South American Restaurant          1
               Spanish Restaurant               110
               Sushi Restaurant                   7
               Tapas Restaurant                  54
               Thai Restaurant                    3
               Theme Restaurant                   1
               Vegetarian / Vegan Restaurant      7
5              American Restaurant                1
               Asian Restaurant                   3
               Fast Food Restaurant               2
               Italian Restaurant                 1
               Mediterranean Restaurant           2
               Restaurant                         4
               Spanish Restaurant                10
               Sushi Restaurant                   1

In [19]:
options_restaurant.sort_values(by=['Cluster_number','Venue'],ascending=True, inplace=True)

The clusters are made by geographical reasons and we use the type of restaurants to select the type less common in each group made geographically.
Besides we are going to detect if in any cluster we have all type of restaurants.


In [20]:
options_restaurant.reset_index(inplace=True)

In [21]:
opt_cluster0=options_restaurant[options_restaurant['Cluster_number']==0]

In [22]:
opt_cluster0=pd.merge(opt_cluster0,summary_categories,on='Venue Category',how='right')

In [25]:
opt_cluster0.rename(columns={'Venue_x':'N of restaurants in cluster', 'Venue_y':'N of restaurants in Madrid'})
opt_cluster0.fillna(0)

,Cluster_number,Venue Category,Venue_x,Venue_y
0,0.0,American Restaurant,1.0,6
1,0.0,Kebab Restaurant,1.0,1
2,0.0,Peruvian Restaurant,1.0,2
3,0.0,Thai Restaurant,1.0,4
4,0.0,Vegetarian / Vegan Restaurant,1.0,8
5,0.0,Asian Restaurant,2.0,14
6,0.0,Indian Restaurant,2.0,7
7,0.0,Mediterranean Restaurant,2.0,25
8,0.0,Mexican Restaurant,2.0,12
9,0.0,Sushi Restaurant,2.0,10


We can appreciate that the Chinese Restaurant is popular in the city but there is no one in the cluster 0.

In [29]:
opt_cluster1=options_restaurant[options_restaurant['Cluster_number']==1]
opt_cluster1

,Cluster_number,Venue Category,Venue
17,1,Asian Restaurant,1
18,1,Restaurant,1
19,1,Spanish Restaurant,3


In [32]:
opt_cluster1=pd.merge(opt_cluster1,summary_categories,on='Venue Category',how='right')
opt_cluster1
opt_cluster0.rename(columns={'Venue_x':'N of restaurants in cluster', 'Venue_y':'N of restaurants in Madrid'})
opt_cluster0.fillna(0)


,Cluster_number,Venue Category,Venue_x,Venue_y
0,0.0,American Restaurant,1.0,6
1,0.0,Kebab Restaurant,1.0,1
2,0.0,Peruvian Restaurant,1.0,2
3,0.0,Thai Restaurant,1.0,4
4,0.0,Vegetarian / Vegan Restaurant,1.0,8
5,0.0,Asian Restaurant,2.0,14
6,0.0,Indian Restaurant,2.0,7
7,0.0,Mediterranean Restaurant,2.0,25
8,0.0,Mexican Restaurant,2.0,12
9,0.0,Sushi Restaurant,2.0,10


We can appreciate that the Argentinian and Chinese Restaurant are popular in the city but there is no one in the cluster 1.

In [39]:
opt_cluster2=options_restaurant[options_restaurant['Cluster_number']==2]
opt_cluster2=pd.merge(opt_cluster2,summary_categories,on='Venue Category',how='right')
opt_cluster2.rename(columns={'Venue_x':'N of restaurants in cluster', 'Venue_y':'N of restaurants in Madrid'},inplace=True)
opt_cluster2.fillna(0, inplace=True)
opt_cluster2

,Cluster_number,Venue Category,N of restaurants in cluster,N of restaurants in Madrid
0,2.0,Argentinian Restaurant,1.0,5
1,2.0,Asian Restaurant,1.0,14
2,2.0,Chinese Restaurant,1.0,10
3,2.0,Seafood Restaurant,1.0,19
4,2.0,Fast Food Restaurant,2.0,12
5,2.0,Italian Restaurant,2.0,29
6,2.0,Paella Restaurant,2.0,4
7,2.0,Tapas Restaurant,7.0,70
8,2.0,Spanish Restaurant,8.0,158
9,2.0,Restaurant,10.0,105


We can appreciate that the Mediterranean and Sushi Restaurant are popular in the city but there is no one in the cluster 2.

In [46]:
opt_cluster4=options_restaurant[options_restaurant['Cluster_number']==4]
opt_cluster4=pd.merge(opt_cluster4,summary_categories,on='Venue Category',how='right')
opt_cluster4.rename(columns={'Venue_x':'N of restaurants in cluster', 'Venue_y':'N of restaurants in Madrid'},inplace=True)
opt_cluster4.fillna(0, inplace=True)
opt_cluster4

,Cluster_number,Venue Category,N of restaurants in cluster,N of restaurants in Madrid
0,4.0,Brazilian Restaurant,1.0,1
1,4.0,Cuban Restaurant,1.0,1
2,4.0,Falafel Restaurant,1.0,1
3,4.0,Korean Restaurant,1.0,1
4,4.0,Latin American Restaurant,1.0,1
5,4.0,Persian Restaurant,1.0,1
6,4.0,Peruvian Restaurant,1.0,2
7,4.0,Portuguese Restaurant,1.0,1
8,4.0,Ramen Restaurant,1.0,1
9,4.0,South American Restaurant,1.0,1


In this case the one there has no presence is the Kebab Restaurant

In [47]:
opt_cluster5=options_restaurant[options_restaurant['Cluster_number']==5]
opt_cluster5=pd.merge(opt_cluster5,summary_categories,on='Venue Category',how='right')
opt_cluster5.rename(columns={'Venue_x':'N of restaurants in cluster', 'Venue_y':'N of restaurants in Madrid'},inplace=True)
opt_cluster5.fillna(0, inplace=True)
opt_cluster5

,Cluster_number,Venue Category,N of restaurants in cluster,N of restaurants in Madrid
0,5.0,American Restaurant,1.0,6
1,5.0,Italian Restaurant,1.0,29
2,5.0,Sushi Restaurant,1.0,10
3,5.0,Fast Food Restaurant,2.0,12
4,5.0,Mediterranean Restaurant,2.0,25
5,5.0,Asian Restaurant,3.0,14
6,5.0,Restaurant,4.0,105
7,5.0,Spanish Restaurant,10.0,158
8,0.0,Argentinian Restaurant,0.0,5
9,0.0,Brazilian Restaurant,0.0,1


For this cluster the most addecuate should be a tapas bar, so there is no and in the city is really common.

# Conclusions

In summary the recommendations are:

- Cluster 0: Chinese Restaurant
- Cluster 1: Chinese and Argentinian restaurant
- Cluster 2: Mediterranean and sushi restaurant
- cluster 4: Kebab Restaurant
- cluster 5: Tapas restaurant